## Scraping

There is a lot of great data out on the web. Unfortunately, it is not all readily available via APIs. And even when APIs are available, it may restrict the data we have access to. Scraping usually refers to extracting web page content when APIs are not available. 

In the API section, we used urllib to call an API and save data. We can also use it to aid in our extraction of data from webpages.

In [ ]:
import urllib

In [ ]:
html = urllib.urlopen("http://xkcd.com/1481/")
print(html.read())

We can use the urlretrieve function to retrieve a specific resources, such as a file, via url. This is basic web scraping.

If we look through our html above, we can see there is a url for the image in the page. (Look for: ```Image URL (for hotlinking/embedding): https://imgs.xkcd.com/comics/api.png```)

But before we go doing that, maybe we should check the robots.txt file first...

In [ ]:
robot = urllib.urlopen("https://xkcd.com/robots.txt")
print(robot.read())

Looks like we are good!

In [ ]:
urllib.urlretrieve("http://imgs.xkcd.com/comics/api.png", "api.png")

The cell below this is markdown. Double-click on it so it is in editing mode, then execute it to display the file you downloaded with the previous command.

![](api.png)

Using these methods, we are treating the html as an unstructured string. If we want to retrieve the structured markup, we can use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). "Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work."

Let's look at [this page](https://litemind.com/best-famous-quotes). What if we wanted to extract the quotes and authors? First, are we allowed to?

In [ ]:
robot = urllib.urlopen("https://litemind.com/robots.txt")
print(robot.read())

The page we are scraping isn't excluded in the robots.txt file. Let's see what Beautiful Soup can do.

In [ ]:
from bs4 import BeautifulSoup
url = "https://litemind.com/best-famous-quotes"

html = urllib.urlopen(url).read()
soup = BeautifulSoup(html,"html.parser")
print(soup.prettify())

In the cell above, we read our web page with urllib (we can also use the [requests](http://docs.python-requests.org/en/master/) library), then parsed with with the Beautiful Soup html parser. You can read about the different parser option [here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#specifying-the-parser-to-use).

Our parsed data is now in a variable called "soup". We used the ["prettify"](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) method to print something a little more readable. Beautiful Soup has represented the html document as a nested data structure that we can navigate.

Beautiful Soup lets you access information through tags in the html. The tags are the same as the ones in the document. 

In [ ]:
soup.title

Tags have names.

In [ ]:
soup.title.name

Sometimes they have attributes too. 

In [ ]:
soup.title.attr

But title does not. It does contain a string though.

In [ ]:
soup.title.string

We can look at just the head of the page.

In [ ]:
soup.head

Or the body.

In [ ]:
soup.body

If we look through the body, we can see our quotes are contained here, starting after 
```<h2>Wisdom Quotes</h2>```


In [ ]:
soup.h2

In [ ]:
soup.h2.text

Tags have attributes that allow us to [navigate](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#navigating-the-tree) through the structure of the document as well. We can navigate up and down a document's structure by looking at a tag's child and parent attributes. 

In [ ]:
soup.body.parent

In [ ]:
soup.head.parent

We can go "sideways" in a document to look at tags at the same level using sibling. Here we can see that head and body are at the same level in our document.

In [ ]:
soup.head.next_sibling

The structure of your document will determine which of these attributes are available.

As we saw above, the quotes we want to scrape start after the second heading.

In [ ]:
soup.h2.next_sibling

We can chain our attributes to continue accessing things. 

In [ ]:
soup.h2.next_sibling.next_sibling

In [ ]:
soup.h2.next_sibling.next_sibling.next_sibling

That seems a bit cumbersome though, right?

Beautiful Soup also allows us to [search](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree) our document. A common task is to pull all of the URLs linked on a page.

In [ ]:
soup.find('a')

In [ ]:
soup.find_all('a')

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

We found our quotes before using:
```soup.h2.next_sibling.next_sibling.next_sibling```

We can also pull them out using find.

In [ ]:
soup.find('div', class_='wp_quotepage')

And we can pull them out yet another way by using [CSS Selectors](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors).

In [ ]:
soup.select('.wp_quotepage')

Once we have the elements we are looking for, we can write some code to pull them out.

In [ ]:
for quote in soup.select('.wp_quotepage'):
    text = quote.findChildren()[0].renderContents()
    author = quote.findChildren()[1].renderContents()
    print(text, author)

It still isn't perfect, but you can clean it up from there. 

There are a lot of resources out there for building scrapers. Do you have a page you want to scrape? If so, try it out now. We are here to answer your questions so give this a try. If you want some more ideas, here are some resources to take a look at:

**More Examples**
* [Scotch Notebook](https://github.com/nd1/pycon_2017/blob/master/scraping/scotch.ipynb) - This notebook shows the process I went through to scrape a site. It is not a polished tutorial, but instead shows some of my thought process when I am scraping.
* Tutorial for [building your first scraper](http://first-web-scraper.readthedocs.io/en/latest/)
* [Python Web Scraping Tutorial using BeautifulSoup](https://www.dataquest.io/blog/web-scraping-tutorial-python/)
* [Scraping Marvel Comics](http://blog.nycdatascience.com/student-works/scraping-marvel-comics/)
* [Scraping for Craft Beers: A Dataset Creation Tutorial](http://blog.kaggle.com/2017/01/31/scraping-for-craft-beers-a-dataset-creation-tutorial/)

**Things to scrape**:
Wikipedia has a lot of good lists to practice on like [Billboard Year-End Hot 100 singles of 1960](https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1960), [List of whisky distilleries in Scotland](https://en.wikipedia.org/wiki/List_of_whisky_distilleries_in_Scotland), or [List of highest-grossing Indian films](https://en.wikipedia.org/wiki/List_of_highest-grossing_Indian_films) among [other things](https://en.wikipedia.org/wiki/List_of_lists_of_lists).
